In [1]:
import langchain
import pandas as pd

In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
ruta_pdf = './War and Peace (Leo Tolstoy).pdf'

In [4]:
loader = PyPDFLoader(ruta_pdf)
pages = loader.load_and_split()
libro = [page.page_content for page in pages]

In [5]:
meta_datos = pd.read_csv('war_and_peace.csv')

    ARMADO
   

In [6]:
retrieval_ = ['What I say is widen the scope of our society, let the\nmot d’ordre be not virtue alone but independence and action as well!”\n\nNicholas, who had left his nephew, irritably pushed up an armchair, sat\ndown in it, and listened to Pierre, coughing discontentedly and frowning\nmore and more.\n\n“But action with what aim?” he cried. “And what position will you adopt\ntoward the government?”\n\n“Why, the position of assistants. The society need not be secret if the\ngovernment allows it. ', 'Princess Mary will take her there and\nshow her over, and they’ll talk nineteen to the dozen. That’s\ntheir woman’s way! I am glad to have her. Sit down and talk. About\nMikhelson’s army I understand—Tolstóy’s too... a simultaneous\nexpedition.... But what’s the southern army to do? Prussia is\nneutral... I know that. What about Austria?” said he, rising from his\nchair and pacing up and down the room followed by Tíkhon, who ran after\nhim, handing him different articles of clothing. “What of Sweden? ', '“He seeks only for peace, and only\nthese people sans foi ni loi * can give it him—people who recklessly\nhack at and strangle everything—Magnítski, Arakchéev, and tutti\nquanti.... You will agree that if you did not look after your estates\nyourself but only wanted a quiet life, the harsher your steward was the\nmore readily your object might be attained,” he said to Nicholas.\n\n* Without faith or law.\n\n“Well, what does that lead up to?” said Nicholas.\n\n“Well, everything is going to ruin! ', 'Such is the fate not of great men (grands hommes) whom the Russian mind\ndoes not acknowledge, but of those rare and always solitary individuals\nwho, discerning the will of Providence, submit their personal will to\nit. The hatred and contempt of the crowd punish such men for discerning\nthe higher laws.\n\nFor Russian historians, strange and terrible to say, Napoleon—that most\ninsignificant tool of history who never anywhere, even in exile, showed\nhuman dignity—Napoleon is the object of adulation and enthusiasm; he\nis grand. ', 'At that moment it seemed to\nhim that he was chosen to give a new direction to the whole of Russian\nsociety and to the whole world.\n\n“I only wished to say that ideas that have great results are always\nsimple ones. My whole idea is that if vicious people are united and\nconstitute a power, then honest folk must do the same. Now that’s simple\nenough.”\n\n“Yes.”\n\n“And what were you going to say?”\n\n“I? Only nonsense.”\n\n“But all the same?”\n\n“Oh nothing, only a trifle,” said Natásha, smiling still more brightly.\n']

In [7]:
#Limpiar el retrieval de caracteres distractores ****
def limpiar_texto(list_of_retrieval):
    texto_limpio = []
    for texto in list_of_retrieval:
        texto_sin_saltos = texto.replace("\\n", " ")
        texto_limpio_ = texto_sin_saltos.replace("\\", " ")
        texto_limpio_ = texto_limpio_.replace("\n", " ")
        texto_limpio.append(texto_limpio_)
    return texto_limpio

In [8]:
#Buscar cada retrieval en el PDF 
def buscar_pagina(texto_buscado, lista):
    for indice, texto in enumerate(lista):
        if texto_buscado in texto:
            return indice + 1
    return -1

In [56]:
test = []
for texto in limpiar_texto(retrieval_):
    texto_in = texto[1:20]
    test.append(buscar_pagina(texto_in, libro))
test

[1317, 109, 1317, 1222, 1327]

In [38]:
def book_chapter(page,texto_limpio):
    metadatos = meta_datos[meta_datos.apply(lambda row: row['started_page'] < int(page) < row['end_page'], axis=1)]
    if not metadatos.empty:
        chapter = metadatos['Chapter'].iloc[0]
        book = metadatos['Book'].iloc[0] 
        return page, chapter, book
    else:
        n_texto = tmp.index(page)
        p_chapter = libro[page-1].find('CHAPTER')
        p_texto = libro[page-1].find(texto_limpio[n_texto])
        
        if p_chapter > p_texto:
            metadatos = meta_datos[meta_datos['end_page'] == page]
            if not metadatos.empty:
                chapter = metadatos['Chapter'].iloc[0]
                book = metadatos['Book'].iloc[0]
                return page, chapter, book
        else:
            metadatos = meta_datos[meta_datos['started_page'] == page]
            if not metadatos.empty:
                chapter = metadatos['Chapter'].iloc[0]
                book = metadatos['Book'].iloc[0]
                return page, chapter, book
        return None, None, page

In [77]:
test2 = []
for page in test:
    test2.append(book_chapter(page,limpiar_texto(retrieval_)))
test2

[(1317, 'chapter 14', '1st epilogue: 1813 - 20'),
 (109, 'chapter 26', '1st book: 1805'),
 (1317, 'chapter 14', '1st epilogue: 1813 - 20'),
 (1222, 'chapter 4', '15th book: 1812 - 13'),
 (1327, 'chapter 16', '1st epilogue: 1813 - 20')]

In [58]:
def contexto_a_mostrar(metadatos):
    step1 = set(metadatos)
    step2 = sorted(step1, key=lambda x: x[0])
    return step2

In [78]:
contexto_a_mostrar(test2)

[(109, 'chapter 26', '1st book: 1805'),
 (1222, 'chapter 4', '15th book: 1812 - 13'),
 (1317, 'chapter 14', '1st epilogue: 1813 - 20'),
 (1327, 'chapter 16', '1st epilogue: 1813 - 20')]

In [61]:
def obtener_capitulo_libro(chapter, book, page):
    return f"{chapter} of {book} (page {page})"

In [79]:
[obtener_capitulo_libro(chapter, book, page) for page, chapter, book in contexto_a_mostrar(test2)[:-1]]

['chapter 26 of 1st book: 1805 (page 109)',
 'chapter 4 of 15th book: 1812 - 13 (page 1222)',
 'chapter 14 of 1st epilogue: 1813 - 20 (page 1317)']

In [30]:
def obtener_contexto(total_pages):
    if len(total_pages) > 1:
        context_pages = [obtener_capitulo_libro(chapter, book, page) for page, chapter, book in total_pages[:-1]]
        contexto_paginas = ", ".join(context_pages)
        last_page = context_pages[-1]
        texto_final = f"You can find more context for these questions on {contexto_paginas} and {last_page}"
    elif len(total_pages) == 1:
        page, chapter, book = total_pages[0]
        texto_final = f"You can find more context for this question on {obtener_capitulo_libro(chapter, book, page)}"
    else:
        texto_final = "There is no specific context for this question"
    return texto_final

In [80]:
obtener_contexto(contexto_a_mostrar(test2))

'You can find more context for these questions on chapter 26 of 1st book: 1805 (page 109), chapter 4 of 15th book: 1812 - 13 (page 1222), chapter 14 of 1st epilogue: 1813 - 20 (page 1317) and chapter 14 of 1st epilogue: 1813 - 20 (page 1317)'

In [82]:
def funcion_todo(list_of_retrieval, book, metadatos):
    texto_limpio = limpiar_texto(list_of_retrieval)
    tmp = []
    for texto in texto_limpio:
        texto_in = texto[1:20]
        tmp.append(buscar_pagina(texto_in, book))
    metadatos_paginas = []
    for page in tmp:
        metadatos_paginas.append(book_chapter(page,texto_limpio))
    context1 = contexto_a_mostrar(metadatos_paginas)
    final_answer = obtener_contexto(context1)
    return final_answer

In [83]:
funcion_todo(retrieval_,libro,meta_datos)

'You can find more context for these questions on chapter 26 of 1st book: 1805 (page 109), chapter 4 of 15th book: 1812 - 13 (page 1222), chapter 14 of 1st epilogue: 1813 - 20 (page 1317) and chapter 14 of 1st epilogue: 1813 - 20 (page 1317)'

In [66]:
book_chapter(1222,limpiar_texto(retrieval_))

(1222, 'chapter 4', '15th book: 1812 - 13')

In [49]:
tmp

[]